https://regenerativetoday.com/build-a-complete-neural-network-in-python/

In [1]:
import numpy as np
import pandas as pd
import sys
from matplotlib import pyplot as plt

sys.path.append("..")
from Functions.UNSW_DF import *
X, x_test, y, y_test = DF_XY()

( 1 ) Reading Preprocessed CSV files..
	 Training dataset loaded..
	 Testing dataset loaded..

( 2 ) Loading done, splitting into X and Y..
	 ( 2.1 ) x_train Shape:  	 (175341, 53)
	 ( 2.2 ) y_train Shape:  	 (175341,)
	 ( 2.3 ) x_test Shape:  	 (82332, 53)
	 ( 2.4 ) y_test Shape:  	 (82332,)
( 3 ) Done!
PS! Import with: x_train, x_test, y_train, y_test = XY_import()


In [32]:
X = X[:1000]
y = y[:1000]

In [33]:
y_arr = y.unique()#Output:
y_arr

array([0], dtype=int64)

In [34]:
def randInitializeWeights(Lin, Lout):
    epi = (6**1/2) / (Lin + Lout)**0.5
    w = np.random.rand(Lout, Lin)*(2*epi) -epi
    return w

In [35]:
# TODO: Check whether it needs to be transposed
hidden_layer = 25
output = 1
theta1 = randInitializeWeights(len(X.T), hidden_layer)
theta2 = randInitializeWeights(hidden_layer, output)
theta = [theta1, theta2]

In [36]:
theta2

array([[ 0.266627  , -0.42424521,  0.36540084,  0.48449233, -0.32879461,
        -0.35978479,  0.0505856 , -0.42409588,  0.47863813,  0.14473267,
        -0.10813372, -0.45453652, -0.27024204, -0.00253469, -0.1442339 ,
         0.09357443,  0.39276489,  0.0727671 ,  0.13017427, -0.11847352,
         0.34279738,  0.43480677,  0.20187028, -0.16006506, -0.16297758]])

In [37]:
b1 = np.random.randn(25,)
b2 = np.random.randn(1,)

In [38]:
def z_calc(X, theta):
    return np.dot(X, theta.T)

def sigmoid(z):
    return 1/(1+ np.exp(-z))

In [39]:
z1 = z_calc(X, theta1) + b1
a1 = sigmoid(z1)

In [41]:
z2 = z_calc(a1, theta2) + b2
a2 = sigmoid(z2)

In [43]:
l = 3  #the number of layers
b = [b1, b2]
def hypothesis(df, theta):
    a = []
    z = []
    for i in range (0, l-1):
        z1 = z_calc(df, theta[i]) + b[i]
        out = sigmoid(z1)
        a.append(out)
        z.append(z1)
        df = out
    return out, a, z

In [44]:
y1 = np.zeros([len(X), len(y_arr)])
y1 = pd.DataFrame(y1)
for i in range(0, len(y_arr)):
    for j in range(0, len(y1)):
        if y[0][j] == y_arr[i]:
            y1.iloc[j, i] = 1
        else:
            y1.iloc[j, i] = 0
y1.head()

IndexError: invalid index to scalar variable.

In [47]:
del3 = y1-a2

In [48]:
del2 = np.dot(del3, theta2) * a1*(1 - a1)

In [49]:
def sigmoid_grad(z):
    return sigmoid(z)*(1 - sigmoid(z))

In [50]:
theta1 = np.dot(del2.T, pd.DataFrame(a1)) * 0.003
theta2 = np.dot(del3.T, pd.DataFrame(a2)) * 0.003

In [51]:
def cost_function(y, y_calc, l):
    return (np.sum(np.sum(-np.log(y_calc)*y - np.log(1-y_calc)*(1-y))))/m

In [52]:
m = len(X)
def backpropagation(df, theta, y1, alpha):
    out, a, z = hypothesis(df, theta)
    delta = []
    delta.append(y1-a[-1])
    i = l - 2
    while i > 0:
        delta.append(np.dot(delta[-i], theta[-i])*sigmoid_grad(z[-(i+1)]))
        i -= 1
    theta[0] = np.dot(delta[-1].T, df) * alpha
    for i in range(1, len(theta)):
        theta[i] = np.dot(delta[-(i+1)].T, pd.DataFrame(a[0])) * alpha
    out, a, z = hypothesis(df, theta)
    cost = cost_function(y1, a[-1], 1)
    return theta, cost

In [55]:
theta1 = randInitializeWeights(len(X.T), hidden_layer)
theta2 = randInitializeWeights(hidden_layer, output)
theta = [theta1, theta2]
cost_list = []
for i in range(20):
    theta, cost= backpropagation(X, theta, y1, 0.003)
    cost_list.append(cost)
cost_list

[6.348532272145098e-05,
 0.603704195865673,
 3.571184325718512e-05,
 0.6039757685308886,
 3.5618191310204654e-05,
 0.6039767249598829,
 3.5617857876891684e-05,
 0.6039767283660182,
 3.561785668875407e-05,
 0.6039767283781554,
 3.561785668452186e-05,
 0.6039767283781987,
 3.561785668450665e-05,
 0.6039767283781989,
 3.561785668450654e-05,
 0.6039767283781989,
 3.561785668450654e-05,
 0.6039767283781989,
 3.561785668450654e-05,
 0.6039767283781989]

In [59]:
out, a, z = hypothesis(X, theta)

In [65]:
accuracy= 0
for i in range(0, len(out)):
    for j in range(0, len(out[i])):
        if out[i][j] >= 0.5 and y1.iloc[i, j] == 1:
            accuracy += 1
accuracy/len(X)

0.0